In [ ]:
import os
import pandas as pd
import numpy as np
from rucio.client import Client
from Oracle import Oracle

In [ ]:
rucio = Client()

# Deletion File Load
Previously defined datasets to delete

In [ ]:
df = pd.read_csv('dropping_sorted_spring2023.csv')
df.head()

# Query Construction
The idea is by code to add all the datasets to find. They will be formatted in the delete_datasets section and will be kept by the with. These will be used to get the file names (child names) by joining them with the CONTENTS table, this table is at block level, so it requires a substring and a group by section to have the desired granularity. After that, it is joined with LOCKS and RSES to get all the rules protecting the files and the rse name to make it more readable.

In [ ]:
query_op = """
WITH delete_datasets AS (
{}
),
delete_child_names AS (
    SELECT CHILD_NAME, DATASET   
    FROM cms_rucio_prod.CONTENTS c
    INNER JOIN delete_datasets ON Substr(name, 1, Instr(name, '#') - 1) = delete_datasets.DATASET
    GROUP BY CHILD_NAME, DATASET 
)
SELECT *
FROM (
SELECT dataset, rule_id, rse, RSE_TYPE, count(DISTINCT name) OVER(PARTITION BY dataset, rse) AS FILES_NUMBER, sum(bytes) OVER(PARTITION BY dataset, rule_id, rse) AS FILES_SIZE
FROM (
SELECT dataset, name, rule_id, rse, RSE_TYPE, bytes
FROM delete_child_names d
INNER JOIN cms_rucio_prod.LOCKS l ON d.CHILD_NAME = l.NAME
INNER JOIN cms_rucio_prod.RSES r ON r.ID = l.RSE_ID)
)
GROUP BY dataset, rule_id, rse, RSE_TYPE, FILES_NUMBER,FILES_SIZE
"""

In [ ]:
# Create the text needed from each dataset to be part of the WITH statement
# DUAL is just used as joker it doesn't have any responsability, could be any table
def get_dataset_query(dataset_name):
    return f"SELECT '{dataset_name}' AS DATASET FROM DUAL"

In [ ]:
# Get the query string for each dataset
vectorized_get_dataset_query = np.vectorize(get_dataset_query)
# Concat all the statements with an UNION ALL
dataset = ' UNION ALL '.join(list(vectorized_get_dataset_query(df['dataset'].unique())))
# Add the statement to the SQL Query
query_op = query_op.format(dataset)

# Execute Query

In [ ]:
host = os.environ.get('ORACLE_HOST')
port = os.environ.get('ORACLE_PORT')
service = os.environ.get('ORACLE_SERVICE_NAME')
username = os.environ.get('ORACLE_USERNAME')
password = os.environ.get('ORACLE_PASSWORD')

In [ ]:
dbManager = Oracle(host,port,service,username,password)

In [ ]:
df_rse = dbManager.query(query_op)
df_rse.head()

# Export Results

In [ ]:
df_rse.to_csv('deletion_rules_spring2023.csv',index= False)